In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

### Data 전처리

In [6]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str))) # 중복문자 제거 후 sorting
vocab_size = len(char_vocab) # a p l e ! 총 5개
print(char_vocab)
print('문자 집합의 크기 : {}'.format(vocab_size))

['!', 'a', 'e', 'l', 'p']
문자 집합의 크기 : 5


In [4]:
# hyper parameter
input_size = vocab_size
hidden_size = 5
output_size = 5
learning_late = 0.1

In [8]:
# char 집합 indexing
# dict()는 dsictionary type의 data를 생성 또는 변환한다.
# 문자에 순차적으로 고유한 정수 index 부여 후 dictionary 형태로 저장
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [9]:
# 정수 형태의 결과를 다시 char형태로 볼 수 있도록 index_to_char를 선언
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [10]:
# 입력 data와 label data의 각 문자들을 정수로 맵핑
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [11]:
# 배치 자원 추가
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [12]:
# 입력 시퀀스의 각 문자를 numpy를 이용해 one-hot-vector로 변경
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [18]:
# data를 tensor로 변환
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


### Model 구현

In [14]:
class Net(nn.Module):
    def __init__(self, inputsize, hidden_size, output_size):
        super(Net, self).__init__()
        # RNN 셀 구현
        self.rnn = nn.RNN(input_size, hidden_size, batch_first = True)
        # 출력층 구현
        self.fc = nn.Linear(hidden_size, output_size, bias = True)
        
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [15]:
net = Net(input_size, hidden_size, output_size)

In [16]:
outputs = net(X)
print(outputs.shape) # 3차원 Tensor

torch.Size([1, 5, 5])


In [19]:
# 정확도를 측정할 때는 이를 모두 펼쳐서 계산하게 되는데
# 이때는 view를 사용하여 배치 차원과 시점 차원을 하나로 만든다.

print(outputs.view(-1, input_size).shape) # 2차원 Tensor로 변환

torch.Size([5, 5])


In [20]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_late)

In [22]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    # batch 차원의 제거를 위해 view 사용
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    # 기울기 계산
    loss.backward()
    # optimizer parameter update
    optimizer.step()
    
    # 최종 예측값인 각 time-step 별 5차원 벡터에 대해 가장 높은 값의 index 선택
    result = outputs.data.numpy().argmax(axis = 2)
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.54912531375885 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
1 loss:  1.3508678674697876 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
2 loss:  1.1488734483718872 prediction:  [[4 4 4 4 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppp!
3 loss:  0.9300554394721985 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
4 loss:  0.742878794670105 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
5 loss:  0.5753856897354126 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.42846113443374634 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.3093385696411133 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.2145945131778717 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.14751406013965607 prediction:  [[4 4 3 2 0]] t